In [ ]:
import sys
sys.path.append('./rpn_net')
sys.path.append('./rpn_net/util/faster_rcnn_lib')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import yaml
from train_model import ProposalModel

In [ ]:
with open('example.yaml', 'r') as f:
    doc = yaml.load(f)

In [ ]:
demo = doc['taskname']
exp = doc['experimentname']
q = doc['hyperparams']['num_attentions']
initq = 10000
b= doc['hyperparams']['num_boxes']
X = np.load(demo+'/'+doc['data']['states'])
U = np.load(demo+'/'+doc['data']['deltas'])
model = ProposalModel(dim_input=X.shape[1],num_queries=q, num_boxes=b, dim_u = U.shape[1])
print "Made model!"

In [ ]:
im_file = demo+'/'+doc['data']['images']
imgs = np.load(im_file)
imgs = imgs.reshape((-1, doc['data']['image_height'],doc['data']['image_width'], 3))
weights_file = demo+'/'+exp+'/weights_iter'+str(initq)+'.pkl'
print weights_file
model.assign_weights(weights_file)

In [ ]:
%matplotlib inline
for t in range(30):
    img = imgs[t*20]
    feats, boxes, im = model.preprocess(img, draw=True)
    feed_dict = {model.feat_list: [feats],
                 model.boxes : [boxes],
             }
    img = img[:,:,::-1]
    orig = img.copy()
    if np.max(img) < 2.0:
        img = img*255
    img = img.astype(np.uint8)
    crops = [model.proposer.get_crop(b, img.copy()) for b in boxes]
    with model.graph.as_default():
        arg_boxes = model.sess.run(model.arg_boxes, feed_dict=feed_dict)
    for b in range(boxes.shape[0]):
        model.proposer.draw_box(boxes[b], img, 2)

    probs = model.sess.run(model.probs , feed_dict)
    for b in arg_boxes:
        model.proposer.draw_box(b[0], img, 0)
    plt.show(plt.imshow(img))